In [23]:
import os
import sys
import re
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
from IPython.display import clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Supress RDKit warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


In [10]:
# import txt file
def import_library(path):
    with open(path, 'r') as f:
        file = f.readlines()
    return file

def retrieve_spectra(GNPS_ID, Library):
    ID = str(GNPS_ID).strip()
    # print(ID)
    if ID is None:
        print("Error: No ID provided")
        return None
    if "CCMSLIB" in ID:
        # print("The ID is formatted correctly")
        pass
    else:
        print("Error: The provided ID is not in the correct format or is not a GNPS_ID")
        return None
    GNPS_ID_formatted = "SPECTRUMID=" + ID + "\n"
    GNPS_ID_index = Library.index(GNPS_ID_formatted)
    # print("Spectrum ID index: " + str(GNPS_ID_index))
    begin_ion = (int(GNPS_ID_index) - 18)
    # print("Begin ion index: " + str(begin_ion))
    # Using the index of the GNPS_ID, we can find the beginning of the spectra and the end of the spectra
    # Starting at the GNPS_ID_index, continue iterating through the library to find the "END IONS/n" string
    end_ion = None
    for i in range(GNPS_ID_index, len(Library)):
        if Library[i] == "END IONS\n":
            end_ion = i
            break
    # print("End ion index: " + str(end_ion))
    return Library[begin_ion:end_ion+1]

def retrieve_spectra_fast(GNPS_ID, Library, index):
    ID = str(GNPS_ID).strip()
    # print(ID)
    if ID is None:
        print("Error: No ID provided")
        return None
    if "CCMSLIB" in ID:
        # print("The ID is formatted correctly")
        pass
    else:
        print("Error: The provided ID is not in the correct format or is not a GNPS_ID")
        return None
    GNPS_ID_index = index
    # print("Spectrum ID index: " + str(GNPS_ID_index))
    begin_ion = (int(GNPS_ID_index) - 18)
    # print("Begin ion index: " + str(begin_ion))
    # Using the index of the GNPS_ID, we can find the beginning of the spectra and the end of the spectra
    # Starting at the GNPS_ID_index, continue iterating through the library to find the "END IONS/n" string
    end_ion = None
    for i in range(GNPS_ID_index, len(Library)):
        if Library[i] == "END IONS\n":
            end_ion = i
            break
    # print("End ion index: " + str(end_ion))
    return Library[begin_ion:end_ion+1]

def write_mgf(spectra, GNPS_ID, ion, output_path):
    ID = str(GNPS_ID).strip()
    ion = str(ion).strip()
    # Get the index of the spectrum id entry in the spectra list
    SPECTRUMID_index = spectra.index("SPECTRUMID=" + ID + "\n")
    # Get SMILES entry using regex
    try:
        smiles = re.search(r'SMILES=(.*)\n', spectra[12]).group(1)
        # Get the exact mass of smiles using RDKit
        mol = Chem.MolFromSmiles(smiles)
        exact_mass = Chem.rdMolDescriptors.CalcExactMolWt(mol)
        # Replace the PEPMASS entry with the correct exact mass
        spectra[1] = "PEPMASS=" + ("%.5f" % exact_mass) + "\n"
    except:
        pass
    # Replace the spectrum id entry with the correct spectrum id
    spectra[SPECTRUMID_index] = "SPECTRUMID=" + ID + "\n" + "FEATURE_ID=" + ID + "\n" + "POLARITY=POS\n" + "ION=[M+" + ion + "]+\n"
    # Delete the lines containing 'INCHI=' and 'INCHIAUX=' and "SMILES="
    with open(output_path + "/" + ID + ".mgf", 'w') as f:
        for line in spectra:
            f.write(line)
    return None

def basic_substructure_search(Substructure, Library):
    # Search for the structure in the library in all lines that start with "SMILES="
    # If the structure is found, add index to list
    matches = []
    structures = []
    for i in range(len(Library)):
        if Library[i].startswith("SMILES="):
            if Substructure in Library[i]:
                matches.append(Library[i+6])
                structures.append(Library[i].strip("SMILES=").strip("\n"))
    return matches, structures

def basic_mass_search(Mass, Library, ppm):
    # search for the mass in the library in all lines that start with "PEPMASS="
    # If the mass is found, add index to list
    matches = []
    masses = []
    # search_string = "PEPMASS=" + str(Mass) + "\n"
    ppm_error = (Mass * (ppm/1000000))
    for i in range(len(Library)):
        if Library[i].startswith("PEPMASS="):
            x =float(Library[i].strip("PEPMASS=").strip("\n"))
            if abs(x - Mass) <= ppm_error:
                matches.append(Library[i+17].strip("SPECTRUMID").strip("=").strip("\n"))
                masses.append(x)
    return matches, masses

def basic_name_search(Name, Library, num_matches=10):
    # Search for the Name in the library in all lines that start with "NAME="
    # If the name is found, add index to list
    Matches = []
    GNPS_IDs = []
    # Compile the regex
    Regex = re.compile(Name, re.IGNORECASE)
    # Search the library using list comprehension and regex, stop after the number of matches is reaches the num_matches variable. If num_matches is None, then return all matches
    if num_matches == None:
        Matches_index = [i for i in range(len(Library)) if Library[i].startswith("NAME=") if re.search(Regex, Library[i])]
    else:
        pass
    if num_matches != None:
        Matches_index = []
        for i in range(len(Library)):
            if Library[i].startswith("NAME="):
                if re.search(Regex, Library[i]):
                    Matches_index.append(i)
                    if len(Matches_index) == num_matches:
                        break
    # Get the Names from the matches index
    Matches = [Library[i] for i in Matches_index]
    Matches = [i.strip("NAME").strip("=").strip("\n") for i in Matches]
    # Get the GNPS_IDs from the matches index
    GNPS_IDs = [Library[i+9] for i in Matches_index]
    GNPS_IDs = [i.strip("SPECTRUMID").strip("=").strip("\n") for i in GNPS_IDs]
    return Matches, GNPS_IDs

def display_spectra(mgf_spectra):
    if mgf_spectra == None:
        print("Error: No spectra provided")
        return None
    # Get the mass/intensity pairs from the mgf_spectra
    mass_intensity_pairs = []
    # Starting that the line beginning with SCANS=, iterate through the mgf_spectra and append the mass/intensity pairs to the mass_intensity_pairs list. Stop when the line begins with "END IONS"
    for i in range(len(mgf_spectra)):
        if mgf_spectra[i].startswith("SCANS="):
            for j in range(i+1, len(mgf_spectra)):
                if mgf_spectra[j].startswith("END IONS"):
                    break
                else:
                    mass_intensity_pairs.append(mgf_spectra[j].strip("\n").split(" "))
    # Split each mass_intensity_pair into a mass and intensity by splitting on the tab character
    mass_intensity_pairs = [i[0].split("\t") for i in mass_intensity_pairs]
    # print(mass_intensity_pairs)
    # Make 2 lists of the masses and intensities and convert them to floats
    masses = [float(i[0]) for i in mass_intensity_pairs]
    # print(masses)
    intensities = [float(i[1]) for i in mass_intensity_pairs]
    # print(intensities)
    # Plot the mass_intensity_df using a barplot
    fig, ax = plt.subplots(figsize=(20,10))
    ax.bar(masses, intensities, width=0.8, color='#464141')
    # Retrieve the title from the mgf_spectra
    title = mgf_spectra[9].strip("NAME").strip("=").strip("\n") + "; " + mgf_spectra[18].strip("SPECTRUMID").strip("=").strip("\n") + "; " + mgf_spectra[1].strip("PEPMASS").strip("=").strip("\n") + "m/z"
    ax.set_title("Mass Spectrum of " + title, fontsize=24)
    ax.set_xlabel("Mass (m/z)", fontsize=20)
    ax.set_ylabel("Intensity", fontsize=20)
    # Set the x range
    ax.set_xlim(0, max(masses) + 10)
    # Set the x axis to be in increments of 50 with a tick every 10
    # plt.xticks(np.arange(0, mass_intensity_df["Mass"].max(), 50))
    # Remove the x labels
    # plt.xlabel("")
    plt.show()
    return fig

def display_spectra_interactive(mgf_spectra):
    # Get the mass/intensity pairs from the mgf_spectra
    mass_intensity_pairs = []
    # Starting that the line beginning with SCANS=, iterate through the mgf_spectra and append the mass/intensity pairs to the mass_intensity_pairs list. Stop when the line begins with "END IONS"
    for i in range(len(mgf_spectra)):
        if mgf_spectra[i].startswith("SCANS="):
            for j in range(i+1, len(mgf_spectra)):
                if mgf_spectra[j].startswith("END IONS"):
                    break
                else:
                    mass_intensity_pairs.append(mgf_spectra[j].strip("\n").split(" "))
    # Split each mass_intensity_pair into a mass and intensity by splitting on the tab character
    mass_intensity_pairs = [i[0].split("\t") for i in mass_intensity_pairs]
    # print(mass_intensity_pairs)
    # Make 2 lists of the masses and intensities and convert them to floats
    masses = [float(i[0]) for i in mass_intensity_pairs]
    # print(masses)
    intensities = [float(i[1]) for i in mass_intensity_pairs]
    # print(intensities)
    mass_intensity_df = pd.DataFrame({"Mass": masses, "Intensity": intensities}, columns=["Mass", "Intensity"], dtype=float)
    # print(mass_intensity_df)
    # Display using the plotly express bar chart
    title=mgf_spectra[9].strip("NAME").strip("=").strip("\n") + "; " + mgf_spectra[18].strip("SPECTRUMID").strip("=").strip("\n") + "; " + mgf_spectra[1].strip("PEPMASS").strip("=").strip("\n") + "m/z"
    fig = px.bar(mass_intensity_pairs,
                 x=0,
                 y=1,
                 title=title)
    fig.update_layout(
        xaxis_title="Mass (m/z)",
        yaxis_title="Intensity",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#464141"
        )
    )
    fig.show()

def input_confirm_structure(ID_structure_dict):
    # Display each structure to the user and ask for input. If input is y then return True, else return False
    # If true, add the key value pair to the dictionary to return
    # If false, do not add the key value pair to the dictionary to return
    # If the user enters "exit", exit the program

    # Create a dictionary to store the user input
    confirmed_structures = {}
    # Iterate through the dictionary and display the key value pairs to the user
    for key, value in ID_structure_dict.items():
        # print("GNPS_ID: " + key)
        # print("Structure: " + value)
        # Ask for user input
        # Display the structure to the user using RDKit
        mol = Chem.MolFromSmiles(value)
        img = Chem.Draw.MolToImage(mol)
        display(img)
        question = input("Do you want to add this structure to the dictionary? (y/n): ")
        if question == "exit":
            sys.exit()
        if question == "y" or question == "Y":
            # Add the key value pair to the dictionary to return
            confirmed_structures[key] = value
            pass
        else:
            # Do not add the key value pair to the dictionary to return
            pass
        clear_output(wait=True)
    return confirmed_structures

In [4]:
gnps_library = import_library("C:/Users/nbrittin/Desktop/GNPS Libraries/ALL_GNPS_05-25-23.mgf")
# print(gnps_library[0:10])

In [5]:
output_dir = "C:/Users/nbrittin/Desktop/Fingerprint Chemical Diversity Project/Polyene Identification/GNPS Spectra"
# write_mgf(spectra, "CCMSLIB00000001547", 'H', output_dir)

In [27]:
polyene_matches, polyene_structures = basic_substructure_search("C=C/C=C/C=C", gnps_library)
print(len(polyene_matches))
print(len(polyene_structures))
# Strip the SPECRTUMID= from the polyene_matches
polyene_matches = [i.strip("SPECTRUMID").strip("=").strip("\n") for i in polyene_matches]
# Make dictionary from matches and structures
polyene_dict = dict(zip(polyene_matches, polyene_structures))
print(len(polyene_dict))
# Use the input_confirm_structure function to get the user input on the polyene_dict
confirmed_polyenes = input_confirm_structure(polyene_dict)
print(len(confirmed_polyenes))
print(confirmed_polyenes)
# Remove duplicates from the confirmed_polyenes dictionary using the values
# Create a list of the values
polyene_values = list(confirmed_polyenes.values())
# Create a list of the unique values
unique_polyene_values = list(set(polyene_values))
# Create a dictionary from the unique values list and their matching keys in the confirmed_polyenes dictionary
unique_polyenes = {key: value for key, value in confirmed_polyenes.items() if value in unique_polyene_values}
print(len(unique_polyenes))
print(unique_polyenes)

unique_polyene_mols = []
# Convert all values in the unique_polyenes dictionary to mols and display them
for key, value in unique_polyenes.items():
    mol = Chem.MolFromSmiles(value)
    unique_polyene_mols.append(mol)

img = Chem.Draw.MolsToGridImage(unique_polyene_mols, molsPerRow=4, subImgSize=(200,200))
display(img)
# Remove the 14th entry from the unique_polyenes dictionary
bad = list(unique_polyenes.keys())[13]
del unique_polyenes[bad]
print(len(unique_polyenes))
unique_polyene_mols = []
# Convert all values in the unique_polyenes dictionary to mols and display them
for key, value in unique_polyenes.items():
    mol = Chem.MolFromSmiles(value)
    unique_polyene_mols.append(mol)

img = Chem.Draw.MolsToGridImage(unique_polyene_mols, molsPerRow=4, subImgSize=(200,200))
display(img)

128
128


In [7]:
gnps_ids = []
for i in range(len(gnps_library)):
    if gnps_library[i].startswith("SPECTRUMID="):
        gnps_ids.append(gnps_library[i].strip("SPECTRUMID").strip("=").strip("\n"))
print(len(gnps_ids))

587991


In [9]:
# Make a test set of 1000 random ids
test_set = np.random.choice(gnps_ids, 1000, replace=False)
# Retrieve the spectra for each of the ids in the test_set
test_set_spectra = []
for i in tqdm(range(len(test_set))):
    test_set_spectra.append(retrieve_spectra(test_set[i], gnps_library))
print(len(test_set_spectra))

100%|██████████| 1000/1000 [11:58<00:00,  1.39it/s]

1000


In [11]:
gnps_id_index_dict = {}
for i in range(len(gnps_library)):
    if gnps_library[i].startswith("SPECTRUMID="):
        gnps_id_index_dict[gnps_library[i].strip("SPECTRUMID").strip("=").strip("\n")] = i
print(len(gnps_id_index_dict))

587991


In [20]:
# Make a test dictionary of 1000 random key value pairs
test_set = np.random.choice(list(gnps_id_index_dict.keys()), 1000, replace=False)
test_dict = {}
# Get all the values from the gnps_id_index_dict that match the keys in the test_set
for key in test_set:
    test_dict[key] = gnps_id_index_dict[key]

# Print the first 10 entries of the test_dict
print(list(test_dict.items())[0:10])


[('CCMSLIB00005763202', 63149662), ('CCMSLIB00009949614', 39573638), ('CCMSLIB00005771178', 63538654), ('CCMSLIB00006472812', 55627520), ('CCMSLIB00006485005', 55986350), ('CCMSLIB00000563242', 63969928), ('CCMSLIB00006120256', 93862244), ('CCMSLIB00006455379', 55132908), ('CCMSLIB00006446913', 54863623), ('CCMSLIB00010007800', 32824772)]


In [21]:
# Retrieve the spectra for each of the ids in the test_dict
test_set_spectra = []
for key, value in tqdm(test_dict.items()):
    test_set_spectra.append(retrieve_spectra_fast(key, gnps_library, value))
print(len(test_set_spectra))

100%|██████████| 1000/1000 [00:00<00:00, 71424.02it/s]

1000


In [34]:
# Import csv with all the GNPS IDs and their predictions
predicted_positive = pd.read_csv("C:/Users/nbrittin/Desktop/Polyene_Identification_Project/Experiments/Postgenesus_Model_Evaluation/Testing Models FPR/GNPS_predicted_positives.csv")

# Extract the GNPS IDS from the Name column
predicted_positive["GNPS_ID"] = predicted_positive["Name"].str.extract(r'(CCMSLIB[0-9]+)')
print(predicted_positive.head())

# Make a list of the GNPS IDs
predicted_positive_ids = list(predicted_positive["GNPS_ID"])
print(len(predicted_positive_ids))
    

                                      Name  Positive             GNPS_ID
0   CCMSLIB00004699064_C51H82O21_[M_Na]__6         1  CCMSLIB00004699064
1   CCMSLIB00004705991_C42H72O14_[M_H]__13         1  CCMSLIB00004705991
2   CCMSLIB00004707696_C47H74O17_[M_K]__14         1  CCMSLIB00004707696
3  CCMSLIB00004710151_C28H36N2O4_[M-H]-_17         1  CCMSLIB00004710151
4   CCMSLIB00004721627_C48H72O14_[M_K]__26         1  CCMSLIB00004721627
59


In [44]:
# For each of the ids in the predicted_positive_ids list get the index from the gnps_id_index_dict
predicted_positive_indices_dict = {}
failed_ids = []
print(len(predicted_positive_ids))
for i in tqdm(range(len(predicted_positive_ids))):
    # Try to get the index from the gnps_id_index_dict
    try:
        predicted_positive_indices_dict[predicted_positive_ids[i]] = gnps_id_index_dict[predicted_positive_ids[i]]
    except:
        failed_ids.append(predicted_positive_ids[i])
print(len(predicted_positive_indices_dict), ": The number is less than the number of ids because some ids were duplicates")

59


100%|██████████| 59/59 [00:00<?, ?it/s]

50 : The number is less than the number of ids because some ids were duplicates


In [45]:
# Print the whole dictionary
for key, value in predicted_positive_indices_dict.items():
    print(key, value)

CCMSLIB00004699064 68254546
CCMSLIB00004705991 72098637
CCMSLIB00004707696 73092891
CCMSLIB00004710151 74357500
CCMSLIB00004721627 92566994
CCMSLIB00006374509 52475391
CCMSLIB00006403179 53455917
CCMSLIB00000563309 63973179
CCMSLIB00006488625 56087153
CCMSLIB00000569623 64333397
CCMSLIB00006565210 58842141
CCMSLIB00006685117 92945065
CCMSLIB00009920871 33519229
CCMSLIB00009921138 33585008
CCMSLIB00009922791 34060496
CCMSLIB00009927945 35352050
CCMSLIB00009935582 36949186
CCMSLIB00009936639 37218341
CCMSLIB00009948721 39431850
CCMSLIB00009955842 40874292
CCMSLIB00009958102 41824507
CCMSLIB00009958650 42039538
CCMSLIB00009958923 42129037
CCMSLIB00009963350 43162362
CCMSLIB00009971313 44783354
CCMSLIB00009983018 47405129
CCMSLIB00009987719 48277127
CCMSLIB00009989485 48544172
CCMSLIB00010060712 24843156
CCMSLIB00010062182 24934025
CCMSLIB00010063383 25010231
CCMSLIB00010066750 25259904
CCMSLIB00010068043 25354024
CCMSLIB00010070582 29168062
CCMSLIB00010075344 29423971
CCMSLIB00010075779 2

In [46]:
# Retrieve the spectra for each of the ids in the predicted_positive_indices_dict
predicted_positive_spectra = []
print(len(predicted_positive_indices_dict))
for key, value in tqdm(predicted_positive_indices_dict.items()):
    predicted_positive_spectra.append(retrieve_spectra_fast(key, gnps_library, value))
print(len(predicted_positive_spectra))

50


100%|██████████| 50/50 [00:00<00:00, 50003.62it/s]

50


In [47]:
# Write the spectra to mgf files
for i in tqdm(range(len(predicted_positive_spectra))):
    write_mgf(predicted_positive_spectra[i], predicted_positive_ids[i], 'H', "C:/Users/nbrittin/Desktop/Polyene_Identification_Project/Experiments/Postgenesus_Model_Evaluation/Testing Models FPR/Predicted Positive Spectra")

 46%|████▌     | 23/50 [00:00<00:00, 159.66it/s]


ValueError: 'SPECTRUMID=CCMSLIB00009958923\n' is not in list

In [53]:
print(predicted_positive_spectra[22])

['BEGIN IONS\n', 'PEPMASS=835.534\n', 'CHARGE=1\n', 'MSLEVEL=2\n', 'SOURCE_INSTRUMENT=ESI-qTof\n', 'FILENAME=suspect_list_batch_creation.mgf\n', 'SEQ=*..*\n', 'IONMODE=Negative\n', 'ORGANISM=GNPS-SUSPECTLIST\n', 'NAME=Suspect related to Massbank:LQB00683 PI 36:4 (predicted molecular formula: C43H81O13P) with delta m/z -21.984 (putative explanation: glyoxal-derived AGE (reverse)|Sodium adduct (reverse); atomic difference: -2C,2H|1H,-1Na) [M-H]-\n', 'PI=Pieter Dorrestein\n', 'DATACOLLECTOR=Wout Bittremieux\n', 'SMILES=N/A\n', 'INCHI=1S/C45H79O13P/c1-3-5-7-9-11-13-15-17-18-19-20-22-24-26-28-30-32-34-39(47)57-37(36-56-59(53,54)58-45-43(51)41(49)40(48)42(50)44(45)52)35-55-38(46)33-31-29-27-25-23-21-16-14-12-10-8-6-4-2/h5,7,11,13,17-18,20,22,37,40-45,48-52H,3-4,6,8-10,12,14-16,19,21,23-36H2,1-2H3,(H,53,54)/b7-5-,13-11-,18-17-,22-20- - 21.984 Da\n', 'INCHIAUX=N/A\n', 'PUBMED=N/A\n', 'SUBMITUSER=mpanitchpakdi\n', 'LIBRARYQUALITY=4\n', 'SPECTRUMID=CCMSLIB00009958923\nFEATURE_ID=CCMSLIB000099589

In [51]:
print(predicted_positive_spectra[24][18])

SPECTRUMID=CCMSLIB00009971313



In [52]:
print(predicted_positive_spectra[22][18])

SPECTRUMID=CCMSLIB00009958923
FEATURE_ID=CCMSLIB00009958923
POLARITY=POS
ION=[M+H]+

